<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Recursive_Feature_Elimination" data-toc-modified-id="Recursive_Feature_Elimination-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Recursive_Feature_Elimination</a></span></li><li><span><a href="#Recursive-Feature-Elimination" data-toc-modified-id="Recursive-Feature-Elimination-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Recursive Feature Elimination</a></span><ul class="toc-item"><li><span><a href="#train_test_split-Section" data-toc-modified-id="train_test_split-Section-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>train_test_split Section</a></span></li><li><span><a href="#NOTE-1:" data-toc-modified-id="NOTE-1:-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>NOTE 1:</a></span></li></ul></li><li><span><a href="#MODEL-1:-5-Feature-Linear-Model" data-toc-modified-id="MODEL-1:-5-Feature-Linear-Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>MODEL 1: 5 Feature Linear Model</a></span><ul class="toc-item"><li><span><a href="#Model-1:-5-Feature-Linear-Model" data-toc-modified-id="Model-1:-5-Feature-Linear-Model-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Model 1: 5 Feature Linear Model</a></span></li></ul></li><li><span><a href="#MODEL-2" data-toc-modified-id="MODEL-2-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>MODEL 2</a></span></li></ul></div>

# Life_Expectancy_WHO_UN_Analysis_Modeling

## Recursive_Feature_Elimination

To:&nbsp;&nbsp;&nbsp;&nbsp; Magnimind

From: Matt Curcio, matt.curcio.ri@gmail.com

Date: 2023-01-24

Re:&nbsp;&nbsp;&nbsp; NOTEBOOK #5

---


- **Use** `Clean_LE_Data_FEng_3.csv`

In [8]:
# Common Python Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# import warnings
import warnings
warnings.filterwarnings("ignore")

# Libraries from Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm 

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [9]:
!ls *.csv

Clean_LE_Data_FEng_3.csv      Life_Expectancy_Data.csv	y_test_LE_Splits.csv
Clean_LE_Data_Post_EDA_2.csv  x_test_LE_Splits.csv	y_train_LE_Splits.csv
Clean_LE_Data_w_Means_1.csv   x_train_LE_Splits.csv


In [10]:
# Reality check ;)

df = pd.read_csv("Clean_LE_Data_FEng_3.csv")

df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2928 entries, 0 to 2927
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country         2928 non-null   object 
 1   Year            2928 non-null   int64  
 2   Status          2928 non-null   int64  
 3   LifeExpectancy  2928 non-null   float64
 4   AdultMort       2928 non-null   float64
 5   EtOH            2928 non-null   float64
 6   BMI             2928 non-null   float64
 7   lt5yD           2928 non-null   int64  
 8   Polio           2928 non-null   float64
 9   TotalExpen      2928 non-null   float64
 10  DTP             2928 non-null   float64
 11  HIV             2928 non-null   float64
 12  Thin5_9y        2928 non-null   float64
 13  Income          2928 non-null   float64
 14  Education       2928 non-null   float64
 15  Region          2928 non-null   object 
 16  ge35BMI         2928 non-null   int64  
dtypes: float64(11), int64(4), object(

## Recursive Feature Elimination

### train_test_split Section

In [11]:
x = df.drop(['LifeExpectancy', 'Country', 'Region'], axis=1)
y = df[['LifeExpectancy']]

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.3,
                                                    random_state=100)

# Convert to categorical
df['Status'] = pd.Categorical(df['Status']) 

print('\nShape of x_train is', {x_train.shape})
print('\nShape of y_train is', {y_train.shape})


Shape of x_train is {(2049, 14)}

Shape of y_train is {(2049, 1)}


In [12]:
df.to_csv('x_train_LE_Splits.csv', index=False)
df.to_csv('x_test_LE_Splits.csv', index=False)
df.to_csv('y_train_LE_Splits.csv', index=False)
df.to_csv('y_test_LE_Splits.csv', index=False)

### NOTE 1:
- Cannot Use Stratify, option for 'train_test_split'. In this case, there is not sufficient class labels of one of your classes to keep the data splitting ratio equal to test_size.

In [13]:
scaler = MinMaxScaler()

numerical_vars = ['AdultMort', 'EtOH', 'BMI', 'lt5yD', 'Polio',
                  'TotalExpen', 'DTP', 'HIV', 'Thin5_9y','Income',
                  'Education', 'ge35BMI']
                  
x_train[numerical_vars] = scaler.fit_transform(x_train[numerical_vars])


In [14]:
lm = LinearRegression()
lm.fit(x_train,y_train)
rfe = RFE(lm)
rfe = rfe.fit(x_train, y_train)

In [17]:
feature_importance = list(zip(x_train.columns,rfe.support_,rfe.ranking_))

In [16]:
def Sort_Tuple(tup):
    """ reverse = None (Sorts in Ascending order)
    key is set to sort using second element of
    sublist lambda has been used
    """
    tup.sort(key = lambda x: x[2])
    return tup
 

# printing the sorted list of tuples
Sort_Tuple(feature_importance)

[('AdultMort', True, 1),
 ('lt5yD', True, 1),
 ('Polio', True, 1),
 ('DTP', True, 1),
 ('HIV', True, 1),
 ('Income', True, 1),
 ('Education', True, 1),
 ('Status', False, 2),
 ('ge35BMI', False, 3),
 ('TotalExpen', False, 4),
 ('Thin5_9y', False, 5),
 ('BMI', False, 6),
 ('Year', False, 7),
 ('EtOH', False, 8)]

**NOTE 5: Inference**
    
- Try first model with `Income`, `Education`,`HIV`, `DTP`, `Polio`, `lt5y_D`,`AdultMort`

In [ ]:
x_train.columns[selector.support_]

In [ ]:
https://www.kaggle.com/code/goyalshalini93/car-price-prediction-linear-regression-rfe

## MODEL 1: 5 Feature Linear Model

In [ ]:
# MODEL 1

x_train_new = build_model(x_train_rfe, y_train)

---

### Model 1: 5 Feature Linear Model

$$Life ~Expectancy ~=~ 57.0 ~+~ 10.1*Income ~+~ 18.4*Education ~-~ 24.1*HIV ~-~ 17.7*Adult ~Mortality ~-~ 10.7*lt5y-D$$

---

## MODEL 2

In [ ]:
# MODEL 2

x_train_8 = x_train[['Income', 'Ed', 'HIV', 'A_Mort', 'lt5y_D', 'Dipt', 'BMI', 'Polio']]

print(x_train_8.shape)

x_train_8.head()

x_train_new = build_model(x_train_8, y_train)


In [ ]:
def scatter(df, column, fig):
    plt.subplot(5,2,fig)
    plt.scatter(df[column], df['L_Expec'])
    plt.title(column+' Vs Life Expectancy')
    plt.ylabel('Life Expectancy')
    plt.xlabel(column)

plt.figure(figsize=(11,18))

scatter(df, 'Income', 1)

In [ ]:
scatter(df, 'Ed', 2)

In [ ]:
scatter(df, 'HIV', 3)

In [ ]:
scatter(df, 'A_Mort', 4)

In [ ]:
scatter(df, 'Dipt', 5)

In [ ]:
scatter(df, 'BMI', 6)

In [ ]:
scatter(df, 'Polio', 7)

plt.tight_layout()